In [ ]:
import pandas as pd
import pandas_utils as pu
import rdkit_mol_identifiers as rdkit
from rdkit.Chem import PandasTools

}## Read multiple library files with inchikey column
only inchikey needed

In [ ]:
libraries = {
  "mcebio": r"C:\git\msn_library\data\library\mce_library_all_cleaned.tsv",
  "nihnp": r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv",
  "mcescaf": r"C:\git\msn_library\data\iocb_libraries\MCE\5k_scaffold\mcescaf_cleaned.tsv",
  "otavapep": r"C:\git\msn_library\data\iocb_libraries\iocb_peptide_library_cleaned.tsv",
  "mcedrug": r"C:\git\msn_library\data\iocb_libraries\fda_approved_drugs\mcedrug\mcedrug_cleaned.tsv",
  "enammol": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\enammol_cleaned.tsv",
  "enamdisc": r"C:\git\msn_library\data\iocb_libraries\radim_enamine_10k_diversity\enamdisc_10k_cleaned.tsv",
  "diana1": r"C:\git\msn_library\data\DIANA\diana_plate1_L0152_standardized.tsv",
  "diana2": r"C:\git\msn_library\data\DIANA\Preplated_100K_platesL1618_1632_standardized.tsv",
  "mcediv": r"C:\git\msn_library\data\iocb_libraries\MCE\50k_diversity_set20k\50k_diversity_set_20k_new_headers_standardized.tsv",
  # "gnps": r"C:\git\msn_library\data\gnpslib\20240229_ALL_GNPS_NO_PROPOGATED_standardized.tsv",
  # "fdl_nmr": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\FDL_NMR_lib_cleaned.tsv",
  # "romanas": r"C:\git\msn_library\data\merlin\GIAR-KI lab 2022-10-26 Chemical standards stock solutions for Pluskal lab.tsv",
  # "france": r"C:\git\msn_library\data\merlin\france\CN_France_cleaned.tsv",
  "nist23": r"C:\git\msn_library\data\public_library\NIST23tandemLib_statistics_standardized.csv",
  "nist20": r"C:\git\msn_library\data\public_library\NIST20_standardized.tsv",
  "mzcloud": r"C:\git\msn_library\data\public_library\mzCloud_04.03.2024_for Customer_standardized.tsv",
  # "gpncl": r"C:\git\msn_library\data\compound_libraries\GPNCL.480_smiles_standardized.tsv",
  # "geneva": r"C:\git\msn_library\data\merlin\genf\lsmls-plate-map-sheet.tsv",
  # "warth_xeno_poly": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\Warth_Vienna_Xenobiotics _PoPhe_new_headers_cleaned.tsv",
  # "petras_np_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\Petras_Tübingen_Natural_Product_Library_cleaned.tsv",
  # "koellensperger_mix": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\Koellensperger_compounds_cleaned.tsv",
  "targetmol_np_4533": r"C:\git\msn_library\data\compound_libraries\targetmol\L6000-Targetmol-Natural Product Library for HTS-4533cpds_new_headers_standardized.tsv",
  # "selleckchem_subset": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem_subset_L5000-1w_cleaned.tsv",
  # "selleckchem_np_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L1400-Natural-Product-Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_phenol_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L1410-Natural Phenol Compound Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_terpenoid_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L1420-Natural Terpenoid Compound Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_organic_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L7600-Natural-Organic-Compound-Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_flavonoid_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L7700-Flavonoid-Compound-Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_alkaloid_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L7900-Alkaloid-Compound-Library-96-well_new_headers_cleaned.tsv",
  # "puretitre": r"C:\git\msn_library\data\compound_libraries\raw_data\puretitre_caithnessbiotechnologies_new_header_cleaned.tsv",
  # "analyticon_megx_all": r"C:\git\msn_library\data\compound_libraries\raw_data\MEGx_Release_2023_09_01\Analyticon_MEGx_Release_2023_09_01_All_6510_cleaned.tsv",
  # "analyticon_natx_all": r"C:\git\msn_library\data\compound_libraries\raw_data\NATx_Release_2023_09_01\Analyticon_NATx_Release_2023_09_01_All_33271_cpds_cleaned.tsv", 
  # 
}

acquired = [
  "mcebio", 
  "nihnp",
  "mcescaf",
  "otavapep",
  "mcedrug",
  "enammol",
  "enamdisc"
]

collaborators = [
  "petras_np_library",
  "warth_xeno_poly",
  "koellensperger_mix",
]

In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["inchikey"]].copy()
  # df = pu.read_dataframe(value)[["inchikey", "monoisotopic_mass"]].copy()
  # df = df[df["monoisotopic_mass"] > 114].drop(columns=["monoisotopic_mass"])
  df = df.dropna(subset="inchikey").drop_duplicates(["inchikey"]).set_index(["inchikey"])
  df[key] = True
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)
  
  
merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)
merged_df["split_inchikey"] = [rdkit.split_inchikey(inchikey) for inchikey in merged_df.index]
merged_df["acquired"] = merged_df[acquired].any(axis=1)


merged_df

In [ ]:
merged_df[merged_df["split_inchikey"] == "GOLXRNDWAUTYKT"]

In [ ]:
merged_df[merged_df["diana"] == True]

In [ ]:
merged_df[(merged_df["diana2"] == True) & (merged_df["acquired"] != True)]

In [ ]:
targetmol = merged_df[(merged_df["targetmol_np_4320"] == True) & (merged_df["acquired"] != True)].reset_index()
targetmol
# pu.save_dataframe(new_targetmol, r"C:\git\msn_library\data\compound_libraries\raw_data\L6000-Targetmol-Natural Product Library for HTS-4320cpds_subset_new.tsv")

In [ ]:
new_targetmol = merged_df[(merged_df["targetmol_np_4533"] == True) & (merged_df["acquired"] != True)].reset_index()
new_targetmol
# pu.save_dataframe(new_targetmol, r"C:\git\msn_library\data\compound_libraries\targetmol\L6000-Targetmol-Natural Product Library for HTS-4533cpds_new_headers_standardized_subset_2184new.tsv")

In [ ]:
merged_df_filtered = merged_df[merged_df["entries"] > 1]

In [ ]:
merged_df_filtered

In [ ]:
merged_df_filtered[(merged_df_filtered["mce"] == True) & (merged_df_filtered["nih"] == True)]

In [ ]:
merged_df["collaborators"] = merged_df[collaborators].any(axis=1)
merged_df["accessible"] = merged_df[["acquired", "collaborators"]].any(axis=1)
merged_df

In [ ]:
not_acquired = merged_df.loc[merged_df["acquired"] == False]
not_accessible = merged_df.loc[merged_df["accessible"] == False]

In [ ]:
not_acquired 

In [ ]:
acquired = merged_df.loc[merged_df["acquired"]]
acquired

In [ ]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset", "selleckchem_organic_library", "selleckchem_terpenoid_library", "selleckchem_flavonoid_library", "selleckchem_alkaloid_library", "selleckchem_np_library", "puretitre", "analyticon_megx_all", "analyticon_natx_all", "petras_np_library",  "warth_xeno_poly", "koellensperger_mix", "fdl_nmr"]:
  print(f"Column {col} has {not_acquired[col].sum()} True values.")


In [ ]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset", "selleckchem_organic_library", "selleckchem_terpenoid_library", "selleckchem_flavonoid_library", "selleckchem_alkaloid_library", "selleckchem_np_library", "puretitre", "analyticon_megx_all", "analyticon_natx_all", "petras_np_library",  "warth_xeno_poly", "koellensperger_mix", "fdl_nmr"]:
  print(f"Column {col} has {not_accessible[col].sum()} True values.")

## Same workflow for split_inchikey

In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["split_inchikey", "monoisotopic_mass"]].copy()
  df = df[df["monoisotopic_mass"] > 114].drop(columns=["monoisotopic_mass"])
  df = df.dropna(subset="split_inchikey").drop_duplicates(["split_inchikey"]).set_index(["split_inchikey"])
  df[key] = True
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)

merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)
merged_df["acquired"] = merged_df[acquired].any(axis=1)


merged_df

In [ ]:

# merged_df["collaborators"] = merged_df[collaborators].any(axis=1)
# merged_df["accessible"] = merged_df[["acquired", "collaborators"]].any(axis=1)

not_acquired = merged_df.loc[merged_df["acquired"] == False]
# not_accessible = merged_df.loc[merged_df["accessible"] == False]

In [ ]:
merged_df[merged_df["acquired"] == True]

In [ ]:
merged_df[merged_df["gpncl"] == True]

In [ ]:
merged_df[(merged_df["gpncl"] == True) & (merged_df["acquired"] != True)]

In [ ]:
new_targetmol = merged_df[
  (merged_df["targetmol_np_4533"] == True) & (merged_df["acquired"] != True)].reset_index()
new_targetmol

In [ ]:
targetmol = merged_df[
  (merged_df["targetmol_np_4320"] == True) & (merged_df["acquired"] != True)].reset_index()
targetmol

In [ ]:
not_acquired

In [ ]:
weizmass_new_df = not_acquired[not_acquired["weizmann"] == True]
weizmass_new_df = weizmass_new_df.reset_index().copy()

In [ ]:
pu.save_dataframe(weizmass_new_df, r"C:\git\msn_library\data\weizmann\weizmass_new_split_inchikey.tsv")

In [ ]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset"]:
  print(f"Column {col} has {not_acquired[col].sum()} True values.")


In [ ]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset"]:
  print(f"Column {col} has {not_accessible[col].sum()} True values.")